In [7]:
# Importing libraries
import pandas as pd
import geopandas as gpd
from os import path
from shapely.geometry import Point
import numpy as np
from globalmaptiles import GlobalMercator
from tqdm import tqdm
merc = GlobalMercator()


In [8]:
gpkg = "/home/kylefelipe/mapa_repo/estudos_tematicos/acre_pessoas_simples_quadkey.gpkg"
gdf = gpd.read_file(gpkg)
# pega valore únicos no campo raça
racas = gdf['raca'].unique()
print(racas)

['parda' 'indigena' 'branca' 'amarela' 'preta']


In [9]:
base_path = "/home/kylefelipe/mapa_repo/estudos_tematicos/"
acre_dados = path.join(base_path, "acre_pessoas_simples.gpkg")

sigla_uf = "AC"

In [3]:
gdf_acre = gpd.read_file(acre_dados)


In [ ]:
cols = gdf_acre.columns
display(gdf_acre.dtypes)

In [5]:
# adiciona a coluna quadkey ao geodataframe
def gen_quadkey(row):
    geometry = row["geometry"]
    mx, my = merc.LatLonToMeters(geometry.y, geometry.x)
    tx, ty = merc.MetersToTile(mx, my, 21)
    quadkey = merc.QuadTree(tx, ty, 21)
    return quadkey
# for idx, row in gdf_acre.iterrows():
#     geometry = row["geometry"]
#     mx, my = merc.LatLonToMeters(geometry.y, geometry.x)
#     tx, ty = merc.MetersToTile(mx, my, 21)
#     quadkey = merc.QuadTree(tx, ty, 21)
#     gdf_acre.at[idx, "quadkey"] = quadkey

# gdf_acre["quadkey"] = gdf_acre.apply(gen_quadkey, axis=1)


In [10]:
# abrir o arquivo geopackage acre_Dados por chunks
quadkey_gpkg = acre_dados.replace(".gpkg", "_quadkey.gpkg")
chunk_size = 1000
i = 0


In [ ]:

while True:
    chunk = slice(i, i + chunk_size)
    print(f"Chunk : {i}")
    gdf_iter = gpd.read_file(acre_dados, rows=chunk)
    if gdf_iter.empty:
        break
    gdf_iter["quadkey"] = gdf_iter.apply(gen_quadkey, axis=1)
    mode = "w" if not path.exists(quadkey_gpkg) else "a"
    gdf_iter.to_file(quadkey_gpkg, mode=mode, layer="pessoas", driver="GPKG")
    i += chunk_size

# gpd.read_file(acre_dados, rows=(0, 1000))
# print(f"Chunk : {chunk}")
# # adiciona quadkey
# chunk["quadkey"] = chunk.apply(gen_quadkey, axis=1)
# mode = "w" if not path.exists(quadkey_gpkg) else "a"
# chunk.to_file(quadkey_gpkg, mode=mode, layer="pessoas", driver="GPKG")

    # salvar o chunk no geopackage



In [8]:
# ordena os dados pelo quadkey

gdf = gpd.read_file(quadkey_gpkg, layer="pessoas")

gdf = gdf.sort_values(by="quadkey")
gdf = gdf.reset_index(drop=True)
gdf.to_file(quadkey_gpkg, layer="pessoas", driver="GPKG")

In [11]:
csv_file = quadkey_gpkg.replace(".gpkg", ".csv")
gdf.to_csv("sorted.csv", index=False)

In [ ]:
print(csv_file)

In [11]:
db_config = {
    "user": "postgres",
    "password": "lJm6VaVGrc4DYPAZ8uBmEAQxLNXSxRyc7jDDmbWSZodlsFkulGt6f6lg22yIF5RJ",
    "dbname": "geoportal_backend",
    "host": "localhost",
    "port": "5555",
}

gdf_quadkey = gpd.read_file(quadkey_gpkg, layer="pessoas")

db_uri = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"
table_schema = "public"
table_name = "acre_pessoas"


In [14]:
print(gdf_quadkey.shape)
# print quadkey crs
print(gdf_quadkey.crs)

(733197, 22)
EPSG:4674


In [12]:

import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine

engine = create_engine(
    db_uri.format(**db_config),
)
# testa conexão
chunk_size = 1000
i = 0
while True:
    chunk = slice(i, i + chunk_size)
    print(f"Enviando pro banco o chunk : {i}")
    gdf_iter = gpd.read_file(quadkey_gpkg, rows=chunk)
    if gdf_iter.empty:
        break
    gdf_iter.to_postgis(table_name, engine, schema=table_schema, if_exists="append", index=False)
    i += chunk_size

print("Dados enviados para o banco de dados")

Enviando pro banco o chunk : 0
Enviando pro banco o chunk : 1000
Enviando pro banco o chunk : 2000
Enviando pro banco o chunk : 3000
Enviando pro banco o chunk : 4000
Enviando pro banco o chunk : 5000
Enviando pro banco o chunk : 6000
Enviando pro banco o chunk : 7000
Enviando pro banco o chunk : 8000
Enviando pro banco o chunk : 9000
Enviando pro banco o chunk : 10000
Enviando pro banco o chunk : 11000
Enviando pro banco o chunk : 12000
Enviando pro banco o chunk : 13000
Enviando pro banco o chunk : 14000
Enviando pro banco o chunk : 15000
Enviando pro banco o chunk : 16000
Enviando pro banco o chunk : 17000
Enviando pro banco o chunk : 18000
Enviando pro banco o chunk : 19000
Enviando pro banco o chunk : 20000
Enviando pro banco o chunk : 21000
Enviando pro banco o chunk : 22000
Enviando pro banco o chunk : 23000
Enviando pro banco o chunk : 24000
Enviando pro banco o chunk : 25000
Enviando pro banco o chunk : 26000
Enviando pro banco o chunk : 27000
Enviando pro banco o chunk : 2800